### Нужно написать две программы:

1) Первая генерирует бинарный файл (min 2Гб), состоящий из случайных 32-рязрядных беззнаковых целых чисел (big endian).

2) Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.

Реализуйте две версии  
1. Простое последовательное чтение 
2. Многопоточная + memory-mapped files. 

Сравните время работы. 

In [202]:
import random
import numpy as np
import time

import mmap
import os
import tempfile

import threading
from concurrent.futures import ThreadPoolExecutor, Future

import warnings
warnings.filterwarnings('ignore')

In [203]:
count = 1000000
filename = "task1_data.txt"

def print_res(sum_arr, max_arr, min_arr):
    print(f'Сумма элементов: \t{sum_arr}')
    print(f'Максимальный элемент: \t{max_arr}')
    print(f'Минимальный элемент: \t{min_arr}')

def write(data, filename):
    file = open(filename, "w")
    res = "\n".join([str(n) for n in data])
    file.write(res)
    file.close()
    
def read(filename):
    file = open(filename)
    data = [int(num) for num in file.read().split('\n') if len(num) > 0]
    file.close()
    return data

def time_counter(func):
    start_time = time.clock()
    func()
    print(time.clock() - start_time, "seconds")
    print("---------------------------------------")

### Обычное решение

In [204]:
def simple_sol_numpy():
    # генерация массива
    mas = []
    rng = random.SystemRandom(0)

    for step in range(count):
        mas.append(rng.randint(10**31, 10**32))

    # сохранение в файл через numpy
    array = np.array(mas)
    np.savetxt(filename, array, newline="\n")

    # решение с numpy
    array = np.fromfile(filename)
    print_res(array.sum(), array.max(), array.min())
    
simple_sol_numpy()

Сумма элементов: 	8.950804433803202e-28
Максимальный элемент: 	4.857875059634242e-33
Минимальный элемент: 	2.9666095620860915e-260


In [205]:
def simple_sol_no_numpy_read_write():
    # генерация массива
    mas = []
    rng = random.SystemRandom(0)

    for step in range(count):
        mas.append(rng.randint(10**31, 10**32))

    # сохранение в файл через функцию
    write(mas, filename)

    # решение без numpy, просто запись-чтение
    sum_num, max_num, min_num = 0, 0, 10**32
    array = read(filename)

    for num in array:
        if (num > max_num):
            max_num = num
        if (num < min_num):
            min_num = num
        sum_num += num

    print_res(sum_num, max_num, min_num)
    
simple_sol_no_numpy_read_write()

Сумма элементов: 	55016357419293429938159561241635516716
Максимальный элемент: 	99999985658716155010329428317656
Минимальный элемент: 	10000052078615530310446554003404


In [206]:
def simple_sol_no_numpy_consist():
    # генерация и сохранение последовательно
    rng = random.SystemRandom(0)
    with open(filename, "w") as file:
        for step in range(count):
            file.write(str(rng.randint(10**31, 10**32)) + '\n')

    # без numpy + последовательное чтение
    sum_num, max_num, min_num = 0, 0, 10**32
    mas = []

    file = open(filename)
    for line in file.readlines():
        num = int(line)
        if (num > max_num):
            max_num = num
        if (num < min_num):
            min_num = num
        sum_num += num
    file.close()

    print_res(sum_num, max_num, min_num)
    
simple_sol_no_numpy_consist()

Сумма элементов: 	54952365033605260056985565163124857202
Максимальный элемент: 	99999947510556274231914433369255
Минимальный элемент: 	10000029755380676046176511474445


### Memory-mapped + threads

In [207]:
# многопоточная версия без numpy + memory-mapped files

f_name = 'file_for_memmap.txt'
rng = random.SystemRandom(0)

sum_num = 0
max_num = 0
min_num = 10**32

dig = 32

def no_threads_mmap():
    
    global sum_num
    global max_num
    global min_num
    
    try:    
        # заполняем файл строками       
        with open(f_name, 'w', encoding='utf-8') as f:
            for num in range(1, count):
                f.write(str(rng.randint(10**31, 10**32)))
                # f.write(str(10**32 + num))

        # мапим файл на память
        with open(f_name, 'r+b') as f:
            with mmap.mmap(f.fileno(), length=dig * (count-1), offset=0, access=mmap.ACCESS_WRITE) as mm:
                # print(mm[0:32])
                
                for i in range(0, count-1):
                    num = int(mm[i*dig: (i+1)*dig])
                    
                    if num > max_num:
                        max_num = num
                    if num < min_num:
                        min_num = num
                    
                    sum_num += num
        
        # проверяем изменения
        with open(f_name, 'r', encoding='utf-8') as f:
            # print(f.readline()[0:32])
            pass
    finally:
        #Удалям файл
        os.remove(f_name)
        print_res(sum_num, max_num, min_num)

no_threads_mmap()

Сумма элементов: 	54997306748785615080077539178795007399
Максимальный элемент: 	99999889619548589234605677540977
Минимальный элемент: 	10000179686248842361385737435356


In [208]:
# многопоточная версия без numpy + memory-mapped files

f_name = 'file_for_memmap.txt'
rng = random.SystemRandom(0)

sum_num = 0
max_num = 0
min_num = 10**32

lock_min = threading.Lock()
lock_max = threading.Lock()

dig = 32

def search(array, size):
        
    global sum_num
    global max_num
    global min_num
    
    for i in range(0, size):
        num = int(array[i*dig: (i+1)*dig])
        sum_num += num
        
        if num > max_num:
            max_num = num
        if num < min_num:
            min_num = num
    

def threads_mmap():
    try:    
        # заполняем файл строками       
        with open(f_name, 'w', encoding='utf-8') as f:
            for num in range(1, count):
                f.write(str(rng.randint(10**31, 10**32)))
                # f.write(str(10**32 + num))

        # мапим файл на память
        with open(f_name, 'r+b') as f:
            with mmap.mmap(f.fileno(), length=dig * (count-1), offset=0, access=mmap.ACCESS_WRITE) as mm:
                # print(mm[0:32])
                
                chunk = count // 4
        
                thread1 = threading.Thread(target=search, args=(mm[0: chunk*dig], chunk))
                thread2 = threading.Thread(target=search, args=(mm[chunk*dig: 2*chunk*dig], chunk))
                thread3 = threading.Thread(target=search, args=(mm[2*chunk*dig: 3*chunk*dig], chunk))
                thread4 = threading.Thread(target=search, args=(mm[3*chunk*dig:], chunk))

                thread1.start()
                thread2.start()
                thread3.start()
                thread4.start()

                thread1.join()
                thread2.join()
                thread3.join()
                thread4.join()
        
        # проверяем изменения
        with open(f_name, 'r', encoding='utf-8') as f:
            # print(f.readline()[0:32])
            pass
    finally:
        #Удалям файл
        os.remove(f_name)
        print_res(sum_num, max_num, min_num)

threads_mmap()

Сумма элементов: 	43166163100460461753411199842843410891
Максимальный элемент: 	99999898024374207804803158356089
Минимальный элемент: 	10000049287312004443395383026339


Exception in thread Thread-34:
Traceback (most recent call last):
  File "c:\users\138904\appdata\local\programs\python\python37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\users\138904\appdata\local\programs\python\python37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-208-f55790b25393>", line 22, in search
    num = int(array[i*dig: (i+1)*dig])
ValueError: invalid literal for int() with base 10: b''



In [209]:
# многопоточная версия без numpy + memory-mapped files

f_name = 'file_for_memmap.txt'
rng = random.SystemRandom(0)

sum_num = 0
max_num = 0
min_num = 10**32

lock_min = threading.Lock()
lock_max = threading.Lock()

dig = 32

def search(array, size):
        
    global sum_num
    global max_num
    global min_num
    
    for i in range(0, size):
        num = int(array[i*dig: (i+1)*dig])
        sum_num += num
        
        if num > max_num:
            max_num = num
        if num < min_num:
            min_num = num
    

def threads_executor_mmap():
    try:    
        # заполняем файл строками       
        with open(f_name, 'w', encoding='utf-8') as f:
            for num in range(1, count):
                f.write(str(rng.randint(10**31, 10**32)))
                # f.write(str(10**32 + num))

        # мапим файл на память
        with open(f_name, 'r+b') as f:
            with mmap.mmap(f.fileno(), length=dig * (count-1), offset=0, access=mmap.ACCESS_WRITE) as mm:
                # print(mm[0:32])
                
                chunk = count // 4
                executor = ThreadPoolExecutor(max_workers=4)
                future1 = executor.submit(search, mm[0: chunk*dig], chunk) 
                future2 = executor.submit(search, mm[chunk*dig: 2*chunk*dig], chunk)
                future3 = executor.submit(search, mm[2*chunk*dig: 3*chunk*dig], chunk)
                future4 = executor.submit(search, mm[3*chunk*dig:], chunk)
        
        # проверяем изменения
        with open(f_name, 'r', encoding='utf-8') as f:
            # print(f.readline()[0:32])
            pass
    finally:
        #Удалям файл
        os.remove(f_name)
        print_res(sum_num, max_num, min_num)

threads_executor_mmap()

Сумма элементов: 	10067250021858296377535852896357132831
Максимальный элемент: 	99999954170497355640457015548331
Минимальный элемент: 	10000165008323626846268406586067


### Подсчет времени

In [210]:
print("Обычное решение с numpy:")
time_counter(simple_sol_numpy)

print("Обычное решение без numpy запись-чтение:")
time_counter(simple_sol_no_numpy_read_write)

print("Обычное решение без numpy с последовательным чтением:")
time_counter(simple_sol_no_numpy_consist)

print("Ускоренное решение с mapping:")
time_counter(no_threads_mmap)

print("Ускоренное решение с mapping и потоками:")
time_counter(threads_mmap)

print("Ускоренное решение с mapping и пулом потоков:")
time_counter(threads_executor_mmap)

Обычное решение с numpy:
Сумма элементов: 	8.989849966608852e-28
Максимальный элемент: 	4.85787505967996e-33
Минимальный элемент: 	2.9666095620860915e-260
13.464835099999618 seconds
---------------------------------------
Обычное решение без numpy запись-чтение:
Сумма элементов: 	54956237976048955551827424022883966723
Максимальный элемент: 	99999996883992361951502762168219
Минимальный элемент: 	10000153842164897681468652799532
7.866681799998332 seconds
---------------------------------------
Обычное решение без numpy с последовательным чтением:
Сумма элементов: 	55003727247389875599109427420505382476
Максимальный элемент: 	99999963393363057435718530816535
Минимальный элемент: 	10000028439656604639360131940027
9.646680700003344 seconds
---------------------------------------
Ускоренное решение с mapping:
Сумма элементов: 	99049728745823916899628229702911328862
Максимальный элемент: 	99999954170497355640457015548331
Минимальный элемент: 	10000066906584466854769633787751
7.77167080000072 

Exception in thread Thread-38:
Traceback (most recent call last):
  File "c:\users\138904\appdata\local\programs\python\python37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\users\138904\appdata\local\programs\python\python37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-209-6905df41f4a3>", line 22, in search
    num = int(array[i*dig: (i+1)*dig])
ValueError: invalid literal for int() with base 10: b''



Сумма элементов: 	144006177850475002458115456443125750640
Максимальный элемент: 	99999954170497355640457015548331
Минимальный элемент: 	10000066906584466854769633787751
7.969380199996522 seconds
---------------------------------------
Ускоренное решение с mapping и пулом потоков:
Сумма элементов: 	154504733692454794214493917050013504280
Максимальный элемент: 	99999954170497355640457015548331
Минимальный элемент: 	10000066906584466854769633787751
7.302940099994885 seconds
---------------------------------------


### Прочие попытки

In [62]:
# numpy + memory-mapped files
# здесь больше чем с 16 разрядами не получилось

f_name = 'file_for_memmap_np.txt'
count = 10
rng = random.SystemRandom(0)

def numpy_mapping():
    try:    
        # записываем туда бинарное представление массива numpy      
        with open(f_name, 'wb') as f:
            mas = [rng.randint(0, 10**16) for num in range(count)] 
            arr = np.array(mas, dtype = np.int64)
            f.write(arr.data)
            print(arr)
            
        # мапим файл на память
        with open(f_name, 'r+b') as f:
            with mmap.mmap(f.fileno(), length=count*8, offset=0, access=mmap.ACCESS_WRITE) as mm:
                # восстанавливаем массив
                arr = np.frombuffer(mm, dtype = np.int64)
                print(arr)  
    finally:
        #Удалям файл
        os.remove(f_name)
        
numpy_mapping()

[7249887284225015 3831196678972692  749394605336612 8831801227387921
 4074856573812913 9739797438265470  140386757324942 6849682196043975
  395772335661113 9684499392227464]
[7249887284225015 3831196678972692  749394605336612 8831801227387921
 4074856573812913 9739797438265470  140386757324942 6849682196043975
  395772335661113 9684499392227464]
